<h2>Аналитическое решение задачи о регрессии</h2>


$$
\hat{\beta} = (X^T X)^{-1} X^T Y

$$

Результат - вектор коэффициентов, который наилучшим образом аппроксимирует зависимость между X и Y в смысле минимализации суммы квадратов ошибок. Этот метод известен как метод наименьших квадратов.

<h2>Регуляризация</h2>

Регуляризация — это техника, используемая в машинном обучении для предотвращения переобучения модели. В частности, регуляризация L1 и L2 помогают управлять сложностью модели и улучшать её обобщающую способность путем добавления штрафов к параметрам модели.

Сравнение L1 и L2

 - L2 регуляризация делает все коэффициенты небольшими, но ненулевыми, что полезно для стабилизации модели и борьбы с мультиколлинеарностью.
 - L1 регуляризация устанавливает некоторые коэффициенты в ноль, что делает её эффективным инструментом для выбора признаков и работы с разреженными моделями.
 
Выбор между L1 и L2 зависит от конкретной задачи. Если целью является выбор небольшого числа значимых признаков, предпочтительнее использовать L1. Если важно сохранить все признаки, но контролировать их влияние, лучше подойдет L2.

<h2>Использование тех же моделей для подбора нелинейных зависимостей</h2>

Чтобы применить линейные методы (например, линейную регрессию, Ridge, Lasso) для моделирования нелинейных зависимостей, нужно преобразовать исходные данные в новые признаки, которые будут отражать нелинейность. Например, можно добавить полиномиальные члены, взаимодействия признаков или другие нелинейные преобразования.

Допусти у нас есть два признака X1 и X2. Мы можем создать новые признаки следующим образом:

- Полиномиальные термины x1^2, x2^2, x1, x2
- Логарифмические преобразования: log(x1)
- Другие нелинейные функции: sin(x), cos(x)

Затем мы применяем линейные модели к новым признакам, что позволяет учесть нелинейные зависимости между исходными признаками и целевой переменной.

Этот подход называется **полиномиальной регрессией**, и он широко применяется в машинном обучении для построения сложных моделей на основе простых линейных методов.

<h2>Вводный анализ данных</h2>

In [1]:
import pandas as pd
import numpy as np
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures

In [2]:
data = pd.read_json('train.json')
upper_perc = np.percentile(data['price'], 99)
lower_perc = np.percentile(data['price'], 1)

data = data[(data['price'] < upper_perc) & (data['price'] > lower_perc)]
data = data.reset_index(drop=True)

In [3]:
data['interest_level'] = data['interest_level'].map({'low': 0, 'medium' : 1, 'high' : 2})

In [4]:
data['features'] = data['features'].apply(lambda x: [item.replace('[', '').replace(']', '').replace("'", "").replace('"', '').replace(' ', '') for item in x])

In [5]:
all_features = []
for row in data['features']:
    all_features.extend(row)
feature_counts = Counter(all_features)
top_features = [feature for feature, count in feature_counts.most_common(20)]
top_features

['Elevator',
 'HardwoodFloors',
 'CatsAllowed',
 'DogsAllowed',
 'Doorman',
 'Dishwasher',
 'NoFee',
 'LaundryinBuilding',
 'FitnessCenter',
 'Pre-War',
 'LaundryinUnit',
 'RoofDeck',
 'OutdoorSpace',
 'DiningRoom',
 'HighSpeedInternet',
 'Balcony',
 'SwimmingPool',
 'LaundryInBuilding',
 'NewConstruction',
 'Terrace']

In [6]:
for feature in top_features:
    data[feature] = data['features'].apply(lambda x: int(feature in x))
feature_list = ['bathrooms', 'bedrooms', 'interest_level'] + top_features
X = data[feature_list]
y = data['price']

In [7]:
X = np.array(X)
y = np.array(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21)

<h2>Реализация моделей - линейная регрессия</h2>

In [8]:
class BaseLinearRegression:
    def __init__(self, learning_rate=0.01, epochs=100):
        """
        Базовый класс для всех линейных регрессий.
        :param learning_rate: скорость обучения (по умолчанию 0.01).
        :param epochs: количество эпох обучения (по умолчанию 1000).
        """
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.w = None  # Веса модели
        self.w = None  # Веса модели
    
    def _add_bias(self, X):
        """
        Добавляет столбец единиц к матрице признаков для свободного члена.
        :param X: матрица признаков (n_samples, n_features).
        :return: матрица с добавленным столбцом единиц.
        """
        return np.c_[np.ones((X.shape[0], 1)), X]
    
    def predict(self, X):
        """
        Предсказание значений на основе обученной модели.
        :param X: матрица признаков (n_samples, n_features).
        :return: вектор предсказанных значений (n_samples,).
        """
        if self.w is None:
            raise ValueError("Модель не обучена. Сначала вызовите метод fit.")
        X_b = self._add_bias(X)
        return X_b.dot(self.w)


In [9]:
class MyLinearRegression(BaseLinearRegression):
    def fit(self, X, y):
        """ 
        Обучение модели линейной регрессии без регуляризации.
        :param X: матрица признаков (n_samples, n_features)
        :param y: вектор целевых значение(n_samples)
        """
        X_b = self._add_bias(X)
        self.w = np.linalg.pinv(X_b.T.dot(X_b)).dot(X_b.T).dot(y)

<h2>Реализация регуляризованных моделей — Ridge, Lasso, ElasticNet</h2>

In [10]:
class MyRidgeRegression(BaseLinearRegression):
    def fit(self, X, y, alpha = 0.1):
        """
        Обучение модели Ridge-регрессии (L2-регуляризации).
        :param X: матрица признаков(n_samples, n_features).
        :param y: вектор целевых значений(n_samples)
        :param alpha: коэффициент регуляризации (по умолчанию 0.1)
        """
        X_b = self._add_bias(X)
        self.w = np.zeros(X_b.shape[1])

        for _ in range(self.epochs):
            for i in range(X_b.shape[0]):
                error = y[i] - X_b[i].dot(self.w)
                self.w += self.learning_rate * (error * X_b[i] - alpha * self.w)

class MyLassoRegression(BaseLinearRegression):
    def fit(self, X, y, alpha = 0.1):
        X_b = self._add_bias(X)
        self.w = np.zeros(X_b.shape[1])

        for _ in range(self.epochs):
            for i in range(X_b.shape[0]):
                error = y[i] - X_b[i].dot(self.w)
                l1_grad = np.sign(self.w)
                self.w += self.learning_rate * (error * X_b[i] - alpha * l1_grad)

class MyElasticNet(BaseLinearRegression):
    def fit(self, X, y, alpha = 0.1, rho = 0.5):
        X_b = self._add_bias(X)
        self.w = np.zeros(X_b.shape[1])

        for _ in range(self.epochs):
            for i in range(X_b.shape[0]):
                error = y[i] - X_b[i].dot(self.w)
                l1_grad = np.sign(self.w)
                l2_grad = self.w
                self.w += self.learning_rate * ( error * X_b[i] - alpha * rho * l1_grad - alpha * (1 - rho) * l2_grad)

<h2>Реализация функций MAE, RMSE, R2</h2>

In [11]:
def my_mean_absolute_error(y_train, y_pred):
    """
    Вычисляет среднюю абсолютную ошибку (MAE)
    """
    if len(y_train) != len(y_pred):
        raise ValueError("Переданы некорректные аргументы")
    
    return np.mean(np.abs(np.array(y_train) - np.array(y_pred)))
def my_mean_squared_errror(y_train, y_pred):
    """ 
    Выичсляет корень из средней квадратичной ошибки
    """
    if len(y_train) != len(y_pred):
        raise ValueError("Переданы некорректные аргументы")
    squared_errors = (np.array(y_train) - np.array(y_pred)) ** 2
    mean_squared_error = np.mean(squared_errors)
    return mean_squared_error

def my_r2_score(y_train, y_pred):
    """ 
    Вычисляет коэффициент детерминации R^2
    """
    if len(y_train) != len(y_pred):
        raise ValueError("Переданы некорректные аргументы")
    y_train = np.array(y_train)
    y_pred = np.array(y_pred)
    y_mean = np.mean(y_train)
    ss_res = np.sum((y_train - y_pred) ** 2)
    ss_tot = np.sum((y_train - y_mean)**2)
    r2 = 1 - (ss_res / ss_tot)
    return r2

<h2>Нормализация функций - MinMaxScaler, StandartScaler</h2>

<h3>Нормализация и масштабирование признаков - это процесс прееобразования данных в одинаковый диапазон или с одинаковой дисперсией. Это важно для многих алгоритмов машинного обучения, так как они могут быть чувствительны к различиям в масштабе признаков</h3>

Нормализация обязательна:
- Градиентный спуск. Потому что градиентный спуск использует производные для обновления весов. Если признаки имеют разные масштабы, то шаги градиентного спуска по каждому признаку будут несбалансированными, что замедляет сходимость. ПРИМЕР: признаки [возраст] (от 0 до 100) и [доход] (от 0 до 1000000). Без нормализации доход будет доминировать в градиентном спуске. 
Нормализация необязательна:
- Деревья решений. Деревья решений ( например случайный лес ) работают с порогами разбиения, а не с абсолютными значениями признаков. Масштаб данных не влияет на работу этих алгоритмов 

### Формула MinMaxScaler

Формула для нормализации данных в диапазон $[a, b]$ с использованием MinMaxScaler:

$$
x' = a + \frac{(x - x_{\text{min}}) \cdot (b - a)}{x_{\text{max}} - x_{\text{min}}}
$$

где:
- $x$ — исходное значение признака,
- $x_{\text{min}}$ — минимальное значение признака в наборе данных,
- $x_{\text{max}}$ — максимальное значение признака в наборе данных,
- $x'$ — нормализованное значение,
- $[a, b]$ — целевой диапазон значений (по умолчанию $[0, 1]$).

### Формула StandardScaler

Формула для стандартизации данных с использованием StandardScaler:

$$
x' = \frac{x - \mu}{\sigma}
$$

где:
- $x$ — исходное значение признака,
- $\mu$ — среднее значение признака в наборе данных,
- $\sigma$ — стандартное отклонение признака в наборе данных,
- $x'$ — стандартизированное значение.

In [12]:
class MyMinMaxScaler:
    def __init__(self, feature_range=(0, 1)):
        """
        Инициализация MinMaxScaler.
        
        :param feature_range: кортеж (min, max), задающий новый диапазон значений.
        """
        self.feature_range = feature_range
        self.min_ = None
        self.scale_ = None

    def fit(self, X):
        """
        Вычисление минимальных и максимальных значений для каждого признака.
        
        :param X: массив данных (numpy array).
        """
        X = np.array(X)
        self.min_ = np.min(X, axis=0)
        self.scale_ = (self.feature_range[1] - self.feature_range[0]) / (np.max(X, axis=0) - self.min_)
        self.scale_[np.isinf(self.scale_)] = 1  # Защита от деления на ноль

    def transform(self, X):
        """
        Нормализация данных в заданный диапазон.
        :param X: массив данных (numpy array).
        :return: нормализованный массив.
        """
        if self.min_ is None or self.scale_ is None:
            raise ValueError("Метод fit должен быть вызван перед transform.")
        X_scaled = self.feature_range[0] + (X - self.min_) * self.scale_
        return X_scaled

    def fit_transform(self, X):
        """
        Объединение fit и transform.
        :param X: массив данных (numpy array).
        :return: нормализованный массив.
        """
        self.fit(X)
        return self.transform(X)

In [13]:
class MyStandardScaler:
    def __init__(self):
        """
        Инициализация StandardScaler.
        """
        self.mean_ = None
        self.scale_ = None

    def fit(self, X):
        """
        Вычисление средних значений и стандартных отклонений для каждого признака.
        
        :param X: массив данных (numpy array).
        """
        X = np.array(X)
        self.mean_ = np.mean(X, axis=0)
        self.scale_ = np.std(X, axis=0)
        self.scale_[self.scale_ == 0] = 1  # Защита от деления на ноль

    def transform(self, X):
        """
        Стандартизация данных.
        
        :param X: массив данных (numpy array).
        :return: стандартизованный массив.
        """
        if self.mean_ is None or self.scale_ is None:
            raise ValueError("Метод fit должен быть вызван перед transform.")
        X_scaled = (X - self.mean_) / self.scale_
        return X_scaled

    def fit_transform(self, X):
        """
        Объединение fit и transform.
        
        :param X: массив данных (numpy array).
        :return: стандартизованный массив.
        """
        self.fit(X)
        return self.transform(X)

In [14]:
def evaluate_model(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    mae_train = mean_absolute_error(y_train, y_train_pred)
    mae_test = mean_absolute_error(y_test, y_test_pred)

    rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
    rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))

    r2_train = r2_score(y_train, y_train_pred)
    r2_test = r2_score(y_test, y_test_pred)

    return mae_train, mae_test, rmse_train, rmse_test, r2_train, r2_test

In [15]:
result_MAE = pd.DataFrame(columns=['model', 'train', 'test'])
result_RMSE = pd.DataFrame(columns = ['model', 'train', 'test'])
result_R2 = pd.DataFrame(columns=['model', 'train', 'test'])

In [16]:
linreg = LinearRegression()
mae_train, mae_test, rmse_train, rmse_test, r2_train, r2_test = evaluate_model(linreg, X_train, y_train, X_test, y_test)
result_MAE.loc[len(result_MAE)] = ['Linreg_default', mae_train, mae_test]
result_RMSE.loc[len(result_RMSE)] = ['Linreg_default', rmse_train, rmse_test]
result_R2.loc[len(result_R2)] = ['Linreg_default', r2_train, r2_test]


In [17]:
ridgereg = Ridge()
mae_train, mae_test, rmse_train, rmse_test, r2_train, r2_test = evaluate_model(ridgereg, X_train, y_train, X_test, y_test)
result_MAE.loc[len(result_MAE)] = ['Ridge_default', mae_train, mae_test]
result_RMSE.loc[len(result_RMSE)] = ['Ridge_default', rmse_train, rmse_test]
result_R2.loc[len(result_R2)] = ['Ridge_default', r2_train, r2_test]

In [18]:
lassoreg = Lasso()
mae_train, mae_test, rmse_train, rmse_test, r2_train, r2_test = evaluate_model(lassoreg, X_train, y_train, X_test, y_test)
result_MAE.loc[len(result_MAE)] = ['Lasso_default', mae_train, mae_test]
result_RMSE.loc[len(result_RMSE)] = ['Lasso_default', rmse_train, rmse_test]
result_R2.loc[len(result_R2)] = ['Lasso_default', r2_train, r2_test]

In [19]:
elastreg = ElasticNet()
mae_train, mae_test, rmse_train, rmse_test, r2_train, r2_test = evaluate_model(elastreg, X_train, y_train, X_test, y_test)
result_MAE.loc[len(result_MAE)] = ['ElasticNet_default', mae_train, mae_test]
result_RMSE.loc[len(result_RMSE)] = ['ElasticNet_default', rmse_train, rmse_test]
result_R2.loc[len(result_R2)] = ['ElasticNet_default', r2_train, r2_test]

<h2>Реализация моделей с MinMaxScaler</h2>

In [20]:
minmax = MinMaxScaler()
X_train_min_max = minmax.fit_transform(X_train)
X_test_min_max = minmax.transform(X_test)

In [21]:
linreg = LinearRegression()
mae_train, mae_test, rmse_train, rmse_test, r2_train, r2_test = evaluate_model(linreg, X_train_min_max, y_train, X_test_min_max, y_test)
result_MAE.loc[len(result_MAE)] = ['Linear MinMaxScaler', mae_train, mae_test]
result_RMSE.loc[len(result_RMSE)] = ['Linear MinMaxScaler', rmse_train, rmse_test]
result_R2.loc[len(result_R2)] = ['Linear MinMaxScaler', r2_train, r2_test]

In [22]:
ridgereg = Ridge()
mae_train, mae_test, rmse_train, rmse_test, r2_train, r2_test = evaluate_model(ridgereg, X_train_min_max, y_train, X_test_min_max, y_test)
result_MAE.loc[len(result_MAE)] = ['Ridge MinMaxScaler', mae_train, mae_test]
result_RMSE.loc[len(result_RMSE)] = ['Ridge MinMaxScaler', rmse_train, rmse_test]
result_R2.loc[len(result_R2)] = ['Ridge MinMaxScaler', r2_train, r2_test]

In [23]:
lassoreg = Lasso()
mae_train, mae_test, rmse_train, rmse_test, r2_train, r2_test = evaluate_model(lassoreg, X_train_min_max, y_train, X_test_min_max, y_test)
result_MAE.loc[len(result_MAE)] = ['Lasso MinMaxScaler', mae_train, mae_test]
result_RMSE.loc[len(result_RMSE)] = ['Lasso MinMaxScaler', rmse_train, rmse_test]
result_R2.loc[len(result_R2)] = ['Lasso MinMaxScaler', r2_train, r2_test]

In [24]:
lassoreg= ElasticNet()
mae_train, mae_test, rmse_train, rmse_test, r2_train, r2_test = evaluate_model(lassoreg, X_train_min_max, y_train, X_test_min_max, y_test)
result_MAE.loc[len(result_MAE)] = ['ElasticNet MinMaxScaler', mae_train, mae_test]
result_RMSE.loc[len(result_RMSE)] = ['ElasticNet MinMaxScaler', rmse_train, rmse_test]
result_R2.loc[len(result_R2)] = ['ElasticNet MinMaxScaler', r2_train, r2_test]

<h2>Реализация моделей Standart Scaler</h2>

In [25]:
stand_scaler = StandardScaler()
X_train_scaler = stand_scaler.fit_transform(X_train)
X_test_scaler = stand_scaler.transform(X_test)

In [26]:
linreg = LinearRegression()
mae_train, mae_test, rmse_train, rmse_test, r2_train, r2_test = evaluate_model(linreg, X_train_scaler, y_train, X_test_scaler, y_test)
result_MAE.loc[len(result_MAE)] = ['Linear StandartScaler', mae_train, mae_test]
result_RMSE.loc[len(result_RMSE)] = ['Linear StandartScaler', rmse_train, rmse_test]
result_R2.loc[len(result_R2)] = ['Linear StandartScaler', r2_train, r2_test]

In [27]:
ridgereg = Ridge()
mae_train, mae_test, rmse_train, rmse_test, r2_train, r2_test = evaluate_model(ridgereg, X_train_scaler, y_train, X_test_scaler, y_test)
result_MAE.loc[len(result_MAE)] = ['Ridge StandartScaler', mae_train, mae_test]
result_RMSE.loc[len(result_RMSE)] = ['Ridge StandartScaler', rmse_train, rmse_test]
result_R2.loc[len(result_R2)] = ['Ridge StandartScaler', r2_train, r2_test]

In [28]:
lassoreg = Lasso()
mae_train, mae_test, rmse_train, rmse_test, r2_train, r2_test = evaluate_model(lassoreg, X_train_scaler, y_train, X_test_scaler, y_test)
result_MAE.loc[len(result_MAE)] = ['Lasso StandartScaler', mae_train, mae_test]
result_RMSE.loc[len(result_RMSE)] = ['Lasso StandartScaler', rmse_train, rmse_test]
result_R2.loc[len(result_R2)] = ['Lasso StandartScaler', r2_train, r2_test]

In [29]:
elastreg = ElasticNet()
mae_train, mae_test, rmse_train, rmse_test, r2_train, r2_test = evaluate_model(elastreg, X_train_scaler, y_train, X_test_scaler, y_test)
result_MAE.loc[len(result_MAE)] = ['ElasticNet StandartScaler', mae_train, mae_test]
result_RMSE.loc[len(result_RMSE)] = ['ElasticNet StandartScaler', rmse_train, rmse_test]
result_R2.loc[len(result_R2)] = ['ElasticNet StandartScaler', r2_train, r2_test]

<h2>Модели с избыточной посадкой</h2>

In [30]:
q1 = data['price'].quantile(0.01)
q99 = data['price'].quantile(0.99)
data = data[(data['price'] >= q1) & (data['price'] <= q99)]

feature_list = ['bathrooms', 'bedrooms', 'interest_level']
X = data[feature_list]
y = data['price']

X = np.array(X)
y = np.array(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

poly = PolynomialFeatures(degree=2)
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)

In [31]:
linreg = LinearRegression()
mae_train, mae_test, rmse_train, rmse_test, r2_train, r2_test = evaluate_model(linreg, X_train_poly, y_train, X_test_poly, y_test)
result_MAE.loc[len(result_MAE)] = ['Linreg Polynomial', mae_train, mae_test]
result_RMSE.loc[len(result_RMSE)] = ['Linreg Polynomial', rmse_train, rmse_test]
result_R2.loc[len(result_R2)] = ['Linreg Polynomial', r2_train, r2_test]

In [32]:
ridgereg = Ridge()
mae_train, mae_test, rmse_train, rmse_test, r2_train, r2_test = evaluate_model(ridgereg, X_train_poly, y_train, X_test_poly, y_test)
result_MAE.loc[len(result_MAE)] = ['Ridge Polynomial', mae_train, mae_test]
result_RMSE.loc[len(result_RMSE)] = ['Ridge Polynomial', rmse_train, rmse_test] 
result_R2.loc[len(result_R2)] = ['Ridge Polynomial', r2_train, r2_test]

In [33]:
mae_train, mae_test, rmse_train, rmse_test, r2_train, r2_test = evaluate_model(lassoreg, X_train_poly, y_train, X_test_poly, y_test)
result_MAE.loc[len(result_MAE)] = ['Lasso Polynomial', mae_train, mae_test]
result_RMSE.loc[len(result_RMSE)] = ['Lasso Polynomial', rmse_train, rmse_test]
result_R2.loc[len(result_R2)] = ['Lasso Polynomial', r2_train, r2_test]

In [34]:
mae_train, mae_test, rmse_train, rmse_test, r2_train, r2_test = evaluate_model(elastreg, X_train_poly, y_train, X_test_poly, y_test)
result_MAE.loc[len(result_MAE)] = ['ElasticNet Polynomial', mae_train, mae_test]
result_RMSE.loc[len(result_RMSE)] = ['ElasticNet Polynomial', rmse_train, rmse_test]
result_R2.loc[len(result_R2)] = ['ElasticNet Polynomial', r2_train, r2_test]

<h2>Собственные модели<h2>

In [35]:
mean_train = y_train.mean()
median_train = np.median(y_train)
mean_test = y_test.mean()
median_test = np.median(y_test)

y_pred_train_mean = np.full(len(y_train), mean_train)
y_pred_test_mean = np.full(len(y_test), mean_test)
y_pred_train_median = np.full(len(y_train), median_train)
y_pred_test_median = np.full(len(y_test), median_test)

mae_train_mean = mean_absolute_error(y_train, y_pred_train_mean)
rmse_train_mean = np.sqrt(mean_squared_error(y_train, y_pred_train_mean))
mae_test_mean = mean_absolute_error(y_test, y_pred_test_mean)
rmse_test_mean = np.sqrt(mean_squared_error(y_test, y_pred_test_mean))

mae_train_median = mean_absolute_error(y_train, y_pred_train_median)
rmse_train_median = np.sqrt(mean_squared_error(y_train, y_pred_train_median))
mae_test_median = mean_absolute_error(y_test, y_pred_test_median)
rmse_test_median = np.sqrt(mean_squared_error(y_test, y_pred_test_median))

result_MAE.loc[len(result_MAE)] = ['native_mean', mae_train_mean, mae_test_mean]
result_RMSE.loc[len(result_RMSE)] = ['native_mean', rmse_train_mean, rmse_test_mean]
result_MAE.loc[len(result_MAE)] = ['native_median', mae_train_median, mae_test_median]
result_RMSE.loc[len(result_RMSE)] = ['native_median', rmse_train_median, rmse_test_median]

<h2>Сравнительный анализ написанных классов и функций</h2>

Сравнение реализации MAE, RMSE, R2

In [36]:
def my_evaluate_model(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    mae_train = my_mean_absolute_error(y_train, y_train_pred)
    mae_test = my_mean_absolute_error(y_test, y_test_pred)

    rmse_train = np.sqrt(my_mean_squared_errror(y_train, y_train_pred))
    rmse_test = np.sqrt(my_mean_squared_errror(y_test, y_test_pred))

    r2_train = my_r2_score(y_train, y_train_pred)
    r2_test = my_r2_score(y_test, y_test_pred)

    return mae_train, mae_test, rmse_train, rmse_test, r2_train, r2_test

In [37]:
linreg = MyLinearRegression()
linres = evaluate_model(linreg, X_train, y_train, X_test, y_test)
linres

(708.3734424247251,
 705.7827601782207,
 973.4168858514256,
 967.2937610280286,
 0.5027486196058539,
 0.5256651712305299)

In [38]:
linres2 = my_evaluate_model(linreg, X_train, y_train, X_test, y_test)
linres2

(708.3734424247251,
 705.7827601782207,
 973.4168858514256,
 967.2937610280286,
 0.5027486196058539,
 0.5256651712305299)

Сравнение реализации моделей

In [39]:
def compare_models(my_model, sklearn_model, X, y, test_size=0.2, random_state=42):

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    
    my_model.fit(X_train, y_train)
    sklearn_model.fit(X_train, y_train)
    
    my_predictions = my_model.predict(X_test)
    sklearn_predictions = sklearn_model.predict(X_test)

    def calculate_metrics(y_true, y_pred):
        mse = mean_squared_error(y_true, y_pred)
        mae = mean_absolute_error(y_true, y_pred)
        rmse = np.sqrt(mse)
        return mse, mae, rmse

    my_mse, my_mae, my_rmse = calculate_metrics(y_test, my_predictions)
    sklearn_mse, sklearn_mae, sklearn_rmse = calculate_metrics(y_test, sklearn_predictions)
    print("=== Сравнение моделей ===")

    print("\nMAE (мои предсказания):", my_mae)
    print("MAE (sklearn):", sklearn_mae)
    print("\nRMSE (мои предсказания):", my_rmse)
    print("RMSE (sklearn):", sklearn_rmse)
    print("=========================")

In [40]:
compare_models(MyLinearRegression(), LinearRegression(), X, y)

=== Сравнение моделей ===

MAE (мои предсказания): 705.7827601782207
MAE (sklearn): 705.7827601782204

RMSE (мои предсказания): 967.2937610280286
RMSE (sklearn): 967.293761028029


In [41]:
compare_models(MyRidgeRegression(), Ridge(alpha=0.1), X, y)

=== Сравнение моделей ===

MAE (мои предсказания): 748.6046493703955
MAE (sklearn): 705.7829702165654

RMSE (мои предсказания): 1048.764398651826
RMSE (sklearn): 967.2941923664423


In [42]:
compare_models(MyLassoRegression(), Lasso(alpha=0.1), X, y)

=== Сравнение моделей ===

MAE (мои предсказания): 727.0126086894695
MAE (sklearn): 705.7832244676124

RMSE (мои предсказания): 1017.3243776726925
RMSE (sklearn): 967.3049501147191


In [43]:
compare_models(MyElasticNet(), ElasticNet(alpha=0.1), X, y)

=== Сравнение моделей ===

MAE (мои предсказания): 732.8567733163807
MAE (sklearn): 716.2800971438973

RMSE (мои предсказания): 1029.4152412596436
RMSE (sklearn): 982.9396763740614


Сравнение реализации MinMaxScaler, StandartScaler

In [44]:
my_scaler = MyMinMaxScaler(feature_range=(0, 1))
original_scaler = MinMaxScaler(feature_range=(0, 1))

X_my_scaled = my_scaler.fit_transform(X)
X_original_scaled = original_scaler.fit_transform(X)

print("Моя трансформация:")
print(X_my_scaled)

print("\nОригинальная трансформация (sklearn):")
print(X_original_scaled)

Моя трансформация:
[[0.1   0.125 0.5  ]
 [0.1   0.25  0.   ]
 [0.1   0.25  0.5  ]
 ...
 [0.1   0.125 0.5  ]
 [0.1   0.25  0.5  ]
 [0.1   0.375 1.   ]]

Оригинальная трансформация (sklearn):
[[0.1   0.125 0.5  ]
 [0.1   0.25  0.   ]
 [0.1   0.25  0.5  ]
 ...
 [0.1   0.125 0.5  ]
 [0.1   0.25  0.5  ]
 [0.1   0.375 1.   ]]


In [49]:
my_scaler = MyStandardScaler()
original_scaler = StandardScaler()

X_my_scaled = my_scaler.fit_transform(X)
X_original_scaled = original_scaler.fit_transform(X)

print("Моя трансформация:")
print(X_my_scaled)

print("\nОригинальная трансформация (sklearn):")
print(X_original_scaled)

Моя трансформация:
[[-0.42321298 -0.48341274  1.0175357 ]
 [-0.42321298  0.43323778 -0.60984835]
 [-0.42321298  0.43323778  1.0175357 ]
 ...
 [-0.42321298 -0.48341274  1.0175357 ]
 [-0.42321298  0.43323778  1.0175357 ]
 [-0.42321298  1.34988831  2.64491975]]

Оригинальная трансформация (sklearn):
[[-0.42321298 -0.48341274  1.0175357 ]
 [-0.42321298  0.43323778 -0.60984835]
 [-0.42321298  0.43323778  1.0175357 ]
 ...
 [-0.42321298 -0.48341274  1.0175357 ]
 [-0.42321298  0.43323778  1.0175357 ]
 [-0.42321298  1.34988831  2.64491975]]


<h2>Отображение результатов</h2>

In [46]:
display(result_MAE)

,model,train,test
0,Linreg_default,687.030471,689.010155
1,Ridge_default,687.026383,689.007983
2,Lasso_default,686.456083,688.620260
3,ElasticNet_default,777.012848,785.570801
4,Linear MinMaxScaler,687.030471,689.010155
5,Ridge MinMaxScaler,687.156337,689.224459
6,Lasso MinMaxScaler,686.643240,689.033861
7,ElasticNet MinMaxScaler,1042.968261,1056.130517
8,Linear StandartScaler,687.030471,689.010155
9,Ridge StandartScaler,687.029272,689.009310


In [47]:
display(result_RMSE)

,model,train,test
0,Linreg_default,996.991413,994.581205
1,Ridge_default,996.991419,994.584120
2,Lasso_default,997.169305,994.928445
3,ElasticNet_default,1149.730025,1162.750102
4,Linear MinMaxScaler,996.991413,994.581205
5,Ridge MinMaxScaler,997.038846,994.791755
6,Lasso MinMaxScaler,997.408179,995.481857
7,ElasticNet MinMaxScaler,1466.888691,1489.317723
8,Linear StandartScaler,996.991413,994.581205
9,Ridge StandartScaler,996.991413,994.581590


In [48]:
display(result_R2)

,model,train,test
0,Linreg_default,0.602062,0.616495
1,Ridge_default,0.602062,0.616493
2,Lasso_default,0.601920,0.616227
3,ElasticNet_default,0.470794,0.475841
4,Linear MinMaxScaler,0.602062,0.616495
5,Ridge MinMaxScaler,0.602024,0.616333
6,Lasso MinMaxScaler,0.601729,0.615800
7,ElasticNet MinMaxScaler,0.138556,0.140066
8,Linear StandartScaler,0.602062,0.616495
9,Ridge StandartScaler,0.602062,0.616495


Самая лучшая модель - Lasso MinMaxScaler

Самая стабильная модель - LassoMinMaxScaler